# Model - Scikit-Learn

> ### Imports

In [1]:
import os
import pandas as pd
from iqual import iqualnlp, evaluation, crossval

> ### Load `annotated (human-coded)` and `unannotated` datasets

In [2]:
data_dir         = "../../data"
human_coded_df   = pd.read_csv(os.path.join(data_dir,"annotated.csv"))
uncoded_df       = pd.read_csv(os.path.join(data_dir,"unannotated.csv"))

> ### Split the data into training and test sets

In [3]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(human_coded_df,test_size=0.25)
print(f"Train Size: {len(train_df)}\nTest Size: {len(test_df)}")

Train Size: 7470
Test Size: 2490


> ### Configure training data

In [4]:
### Select Question and Answer Columns
question_col = 'Q_en'
answer_col   = 'A_en'

### Select a code
code_variable = 'marriage'

### Create X and y
X = train_df[[question_col,answer_col]]
y = train_df[code_variable]

### Initiate model

In [57]:
# Step 1: Initiate the model class
iqual_model = iqualnlp.Model()

# Step 2: Add layers to the model
#  Add text columns, and choose a feature extraction model (Available options: scikit-learn, spacy, sentence-transformers, precomputed (picklized dictionary))
iqual_model.add_text_features(question_col,answer_col,model='TfidfVectorizer')

# Step 3: Add a feature transforming layer (optional)
# A. Choose a feature-scaler. Available options: 
# any scikit-learn scaler from `sklearn.preprocessing`
### iqual_model.add_feature_transformer(name='StandardScaler', transformation="FeatureScaler")
# OR
# B. Choose a dimensionality reduction model. Available options:
# - Any scikit-learn dimensionality reduction model from `sklearn.decomposition`
# - Uniform Manifold Approximation and Projection (UMAP) using umap.UMAP (https://umap-learn.readthedocs.io/en/latest/)

### iqual_model.add_feature_transformer(name='PCA', transformation="DimensionalityReduction")

# Step 4: Add a classifier layer
# Choose a primary classifier model  (Available options: any scikit-learn classifier)
iqual_model.add_classifier(name = "LogisticRegression")

# Step 5: Add a threshold layer. This is optional, but recommended for binary classification
iqual_model.add_threshold()

# Step 6: Compile the model
iqual_model.compile()

Pipeline(steps=[('Input',
                 FeatureUnion(transformer_list=[('question',
                                                 Pipeline(steps=[('selector',
                                                                  FunctionTransformer(func=<function column_selector at 0x00000216DD512F70>,
                                                                                      kw_args={'column_name': 'Q_en'})),
                                                                 ('vectorizer',
                                                                  Vectorizer(analyzer='word',
                                                                             binary=False,
                                                                             decode_error='strict',
                                                                             dtype=<class 'numpy.float64'>,
                                                                             encoding='utf-8',
                                                                             env='scikit-learn',
                                                                             input='co...
                                                                             tokenizer=None,
                                                                             use_idf=True,
                                                                             vocabulary=None))]))])),
                ('Classifier',
                 Classifier(C=1.0, class_weight=None, dual=False,
                            fit_intercept=True, intercept_scaling=1,
                            l1_ratio=None, max_iter=100,
                            model='LogisticRegression', multi_class='auto',
                            n_jobs=None, penalty='l2', random_state=None,
                            solver='lbfgs', tol=0.0001, verbose=0,
                            warm_start=False)),
                ('Threshold', BinaryThresholder())])

In [6]:
# Scorig Dict for evaluation
scoring_dict = evaluation.get_scoring_dict(['f1'])

> ### Configure a Hyperparameter Grid for cross-validation + fitting

In [10]:
search_param_config = {
    "Input":{
        "question":{
            "vectorizer":{
                        "model":["TfidfVectorizer"],
                         "env":["scikit-learn"],               
                        "max_features":[1000,2000],
                        "ngram_range":[(1,2)],
                         },
        },
        "answer":{
            "vectorizer":{
                        "model":["TfidfVectorizer"],
                        "env":["scikit-learn"],                
                        "max_features":[1000,2000],
                        "ngram_range":[(1,2)],
                         },                        
        },
    },
        "Classifier":{
            "model":["LogisticRegression"],
            "C":[0.001,0.01, 0.1],
        },
}

CV_SEARCH_PARAMS = crossval.convert_nested_params(search_param_config)

In [11]:
CV_SEARCH_PARAMS

{'Input__question__vectorizer__model': ['TfidfVectorizer'],
 'Input__question__vectorizer__env': ['scikit-learn'],
 'Input__question__vectorizer__max_features': [1000, 2000],
 'Input__question__vectorizer__ngram_range': [(1, 2)],
 'Input__answer__vectorizer__model': ['TfidfVectorizer'],
 'Input__answer__vectorizer__env': ['scikit-learn'],
 'Input__answer__vectorizer__max_features': [1000, 2000],
 'Input__answer__vectorizer__ngram_range': [(1, 2)],
 'Classifier__model': ['LogisticRegression'],
 'Classifier__C': [0.001, 0.01, 0.1]}

> ## Model training:
> Cross-validate over hyperparameters and select the best model

In [12]:
cv_dict = iqual_model.cross_validate_fit(
    X,y,                                # X: Pandas DataFrame of features, y: Pandas Series of labels
    search_parameters=CV_SEARCH_PARAMS, # search_parameters: Dictionary of parameters to use for cross-validation
    cv_method='GridSearchCV',           # cv_method: Cross-validation method to use, options: GridSearchCV, RandomizedSearchCV
    scoring=scoring_dict,               # scoring: Scoring metric to use for cross-validation
    refit='f1',                         # refit: Metric to use for refitting the model
    n_jobs=-1,                          # n_jobs: Number of parallel threads to use  
    cv_splits=3,                        # cv_splits: Number of cross-validation splits
)

In [13]:
print("Average F1 score: {:.3f}".format(cv_dict['avg_test_score']))

Average F1 score: 0.717


### Evaluate model using out sample data (Held out human-coded data)

In [14]:
test_pred = iqual_model.predict(test_df[['Q_en','A_en']])
test_act  = test_df[code_variable].tolist()

f1_score = evaluation.calc_f1_score_from_labels(test_pred,test_act,)
print(f"Out-sample F1-score: {f1_score:.3f}")

Out-sample F1-score: 0.716


### Predict labels for unannotated data

In [15]:
uncoded_df[code_variable+'_pred'] = iqual_model.predict(uncoded_df[['Q_en','A_en']])

In [16]:
dict(uncoded_df[code_variable+"_pred"].value_counts())

{0: 21782, 1: 721}

In [17]:
for idx, row in uncoded_df.loc[(uncoded_df[code_variable+"_pred"]==1),['Q_en','A_en']].sample(5).iterrows():
    print("Q: ",row['Q_en'],"\n","A: ", row['A_en'],sep='')
    print()

Q: So what else do you hope for your son except for you to study?
A: What else is the hope that my eldest child is a girl, so when she grows up, I will marry her. If I had money, I could study well. Now we have to get married. Even then, there is no money, how can I meet the dowry that I want now?

Q: What other dreams do you have about him?
A: There are many dreams. The big boy talks. He will grow up to work and then marry Arky.

Q: What are your hopes for 3 children?
A: One son has grown up, if he can do business, I can take the money from there and get his daughter married and the other one will study.

Q: He said that he would like to shop after finishing his studies. Besides, I want to know what other hopes there are for him!
A: When I grow up, I will run a shop, then I will find a good girl and marry her. These are the hopes of parents.

Q: Bro, do you have any thoughts/thoughts about girls?
A: I will teach the girls. The rest is in God's hands. After completing my studies I will